In [26]:
# def 各种必要环境
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import json
import logging,os,sys,time
from selenium import webdriver
def start():
	##/初始化路径
	chromedriver="/Users/laiyao/Documents/GitHub/MyApp/PythonApplication1/爬虫练习/chromedriver/chromedriver-2"
	##引入selenium，设置Chrome
	from selenium import webdriver
	import selenium
	os.environ["webdriver.chrome.driver"] = chromedriver
	option = webdriver.ChromeOptions()#自定义设置
	option.add_argument('--user-agent=Mozilla/5.0 (Linux; U; Android 2.2.1; zh-cn; HTC_Wildfire_A3333 Build/FRG83D) AppleWebKit/533.1 (KHTML, like Gecko) Version/4.0 Mobile Safari/533.1') #修改浏览器的User-Agent来伪装你的浏览器
	option.add_argument('--process-per-site') #每个站点使用单独进程
	option.add_argument('--lang=zh-CN') #设置语言为简体中文
	##启动浏览器
	driver = webdriver.Chrome(chromedriver,chrome_options=option)
	driver.get('about:version')
	return driver

def openNewTab(driver,url='https://www.sogou.com'):
    js='window.open("'+url+'");'
    driver.execute_script(js)

def quit(driver):
    driver.close()
    driver.quit()
    logging.info(['QUIT success'])
    return



SyntaxError: invalid syntax (<ipython-input-26-261361188ae0>, line 3)

In [21]:
# 初始化数据储存：
df=pd.DataFrame(columns =['日期','ID','账户名称','总支出','现金支出','赠款支出'] ,index=[0])
df = df.append({'日期':'2000-01-01','ID':0,'账户名称':'涉及金额的账户需要除以100000','总支出':0,'现金支出':0,'赠款支出':0}, ignore_index=True)  #忽略索引,往dataframe中插入一行数据
FailSubAccount={}
def loginToutiao():
    # 开浏览器窗口（三个窗口）
    driver=start()
    openNewTab(driver)
    openNewTab(driver)
    # 业务代码：登录（登录需要手工）
    loginURL='https://ad.toutiao.com/pages/login/index.html'
    driver.switch_to.window(driver.window_handles[0])
    driver.get(loginURL)
    return driver
    
def loginSubAccount(driver,subAccountID=0,name='默认名字'):
    # 业务代码：登录子账户（自动）
    subAccountURL='https://ad.toutiao.com/marco/account/account_cutover/?advertiser_id='
    driver.switch_to.window(driver.window_handles[1])
    driver.get(subAccountURL+str(subAccountID))
    time.sleep(3)
    SubAccountData=getSubAccountData(driver)
    if SubAccountData==None:
        global FailSubAccount
        FailSubAccount[subAccountID]=name
    for data in SubAccountData:
        if not subAccountID ==data["advertiser_id"]:
            logging.error(jsonData)
            logging.error('账户错误')
        else:
            global df
            df = df.append({'日期':data["date"],'ID':data["advertiser_id"],'账户名称':name,'总支出':data["cost"],'现金支出':data["cash_cost"] ,'赠款支出':data["reward_cost"]}, ignore_index=True)  #忽略索引,往dataframe中插入一行数据
    pass

def getSubAccountData(driver,start_date="2019-03-01",end_date="2019-03-08"):
    dataURL='https://ad.toutiao.com/overture/cash/get_cash_flow/?page=1&start_date='+start_date+'&end_date='+end_date
    driver.switch_to.window(driver.window_handles[2])
    driver.get(dataURL)
    # 去掉HTML里面乱七八糟的东西，拿到真实的JSON数据
    htmlText=driver.page_source.replace('<html xmlns="http://www.w3.org/1999/xhtml"><head></head><body><pre style="word-wrap: break-word; white-space: pre-wrap;">','').replace('</pre></body></html>','')
    if htmlText[0] =='{':
        jsonData=json.loads(htmlText)
    else:
        logging.error(htmlText)
        logging.error('浏览器获取数据失败')
        return None
    # 验证"status": "success"
    if not jsonData["status"] =="success":
        logging.error(jsonData)
        logging.error('头条系统获取数据失败')
    return jsonData['data']["items"]

In [3]:
# 业务执行（登录头条）
driver=loginToutiao()

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:20: DeprecationWarning: use options instead of chrome_options


In [ ]:
# 数据源来自这个文件：
# import '/Users/laiyao/Documents/成本/投放成本录入准确性核对/今日头条数据/ID-Name-Python.py'

# 业务执行（拿数据）
rundata=liangyuanSubAccount_ID_Name
for k,v in rundata.items():
    loginSubAccount(driver,k,v)
    print(k,v)


#// 需要的数据在 jsonData['data']["items"][0]["cost"]  //总支出
#// 需要的数据在 jsonData['data']["items"][0]["cash_cost"]  //现金支出
#// 需要的数据在 jsonData['data']["items"][0]["reward_cost"]  //赠款支出
#// 需要的数据在 jsonData['data']["items"][0]["date"]  //日期

#// 验证字段： jsonData['data']["items"][0]["advertiser_id"]  //子账户ID

In [ ]:
df.to_csv(path_or_buf='aa.csv')


df

In [24]:
quit(driver)